# Modelling for Dublin Bus Project    --Team ETA 

## With the final base table we got after data cleaning for two months' data 


# Read final table and check it first 

In [1]:
# Import all the packages we need 
import pandas as pd
import numpy as np

In [2]:
# Read data file to data frame 
%time df_all = pd.read_csv('final_table_model.csv',dtype={ 'Journey_Pattern_ID': object})

CPU times: user 37.9 s, sys: 15.9 s, total: 53.8 s
Wall time: 58.4 s


In [3]:
#Check the first 3 rows 
df_all.head(3)

,Journey_Pattern_ID,Distance,Trip_Time,datetime,HourOfDay,day_of_week,midweek,time_bin,cloud,rain,temp,wind
0,00010001,0,0,2012-11-06 12:15:13,12,Tuesday,0,am,6.88,0.0,7.55,16.25
1,00010001,22,360,2012-11-06 12:21:13,12,Tuesday,0,am,6.88,0.0,7.55,16.25
2,00010001,215,397,2012-11-06 12:21:50,12,Tuesday,0,am,6.88,0.0,7.55,16.25


In [4]:
# Check the last 3 rows 
df_all.tail(3)

,Journey_Pattern_ID,Distance,Trip_Time,datetime,HourOfDay,day_of_week,midweek,time_bin,cloud,rain,temp,wind
16852097,084X1002,23071,3197,2013-01-29 09:07:03,9,Tuesday,0,am,7.38,0.03,8.32,10.88
16852098,084X1002,23318,3303,2013-01-29 09:08:49,9,Tuesday,0,am,7.38,0.03,8.32,10.88
16852099,084X1002,23665,3478,2013-01-29 09:11:44,9,Tuesday,0,am,7.38,0.03,8.32,10.88


In [5]:
# Check the column types 
df_all.dtypes

Journey_Pattern_ID     object
Distance                int64
Trip_Time               int64
datetime               object
HourOfDay               int64
day_of_week            object
midweek                 int64
time_bin               object
cloud                 float64
rain                  float64
temp                  float64
wind                  float64
dtype: object

In [6]:
# check the Journey_Pattern_ID unique value size 
pd.unique(df_all.Journey_Pattern_ID ).size

478

In [7]:
# Assign the dataframe to new varible 
df=df_all

# Train Data set with statsmodels linear regression 

In [8]:
# check data frame size 
df.shape

(16852100, 12)

In [9]:
# Prepare the training features and target feature data set 
feature_cols = ['Distance','midweek','HourOfDay','cloud','rain','wind','temp']
X = df[feature_cols]
y = df['Trip_Time']
X.columns

Index(['Distance', 'midweek', 'HourOfDay', 'cloud', 'rain', 'wind', 'temp'], dtype='object')

In [10]:
# Import statsmodels and train the ordinary least squares model with our data set 
# ordinary least squares (OLS)
import statsmodels.formula.api as sm

df_linear = pd.concat([X, y], axis=1)
%time lm = sm.ols(formula = "Trip_Time ~ Distance+midweek+HourOfDay+cloud+rain+wind+temp", data=df_linear).fit()

CPU times: user 21 s, sys: 37.6 s, total: 58.5 s
Wall time: 1min 37s


In [11]:
# statsmodels linear regression parameters 
lm.params

Intercept    330.786784
Distance       0.177387
midweek     -209.282461
HourOfDay     -3.598221
cloud         -8.368503
rain          56.839499
wind          -1.547951
temp           6.686312
dtype: float64

In [12]:
# statsmodels linear regression summary 
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Trip_Time   R-squared:                       0.803
Model:                            OLS   Adj. R-squared:                  0.803
Method:                 Least Squares   F-statistic:                 9.823e+06
Date:                Sun, 20 Aug 2017   Prob (F-statistic):               0.00
Time:                        11:49:58   Log-Likelihood:            -1.3155e+08
No. Observations:            16852100   AIC:                         2.631e+08
Df Residuals:                16852092   BIC:                         2.631e+08
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    330.7868      0.701    471.864      0.000     329.413     332.161
Distance       0.1774   2.15e-05   8260.852      0.000       0.177       0.177
midweek     -209.2825      0.392   -534.538      0.000    -210.050    -208.515
HourOfDay     -3.5982      0.031   -115.980      0.000      -3.659      -3.537
cloud         -8.3685      0.081   -102.972      0.000      -8.528      -8.209
rain          56.8395      0.484    117.333      0.000      55.890      57.789
wind          -1.5480      0.031    -50.699      0.000      -1.608      -1.488
temp           6.6863      0.047    141.752      0.000       6.594       6.779
==============================================================================
Omnibus:                  2502848.874   Durbin-Watson:                   0.046
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         12096719.747
Skew:                           0.645   Prob(JB):                         0.00
Kurtosis:                       6.945   Cond. No.                     6.00e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large,  6e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [14]:
# The prediction of statsmodels linear regression model 
lm_predictions = lm.predict(X)

In [21]:
lm.conf_int()

,0,1
Intercept,329.412808,332.160759
Distance,0.177345,0.177429
midweek,-210.049827,-208.515094
HourOfDay,-3.659028,-3.537414
cloud,-8.527788,-8.209217
rain,55.890033,57.788965
wind,-1.607792,-1.488109
temp,6.593863,6.778762


In [13]:
# MSE: Mean Squared Error
mse=((df_linear.Trip_Time-lm.predict(df_linear))**2).mean()
print("\n Mean Squared Error",mse)


 Mean Squared Error 353074.658828


In [14]:
# MAE:  Mean Absolute error  
mae = abs(df_linear.Trip_Time-lm.predict(df_linear)).mean()
print("Mean Absolute error  ",mae)

Mean Absolute error   436.961716179


# Result of statsmodels 

In the result we could see that the mean absolute error of the model is 436.961716179
In the summary, the p-value of all the features are equal to 0, it shows that all the features we are using are related to the target feature. 

In those weather feature, the rain feature has the biggest coef, in the final modeling, as the data set size, we hope to use less features and only keep the most important one, so we only choose the rain feature in the modeling at end. 


# Use LinearRegression to train data set without weather feature 

In [15]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [16]:
# Prepare the train data set 
feature_cols = ['Distance','midweek','HourOfDay',]
X = df[feature_cols]
y = df['Trip_Time']
X.columns

Index(['Distance', 'midweek', 'HourOfDay'], dtype='object')

In [17]:
# Train the data set with PolynomialFeatures when degree = 2 linear regression 
polynomial_features = PolynomialFeatures(degree=2,include_bias=False)
linear_regression = LinearRegression()
pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
%time pipeline.fit(X, y)

#df.plot(kind='scatter', x='Distance', y='Trip_Time',label="Samples")
#plt.plot(X['Distance'], pipeline.predict(X), c='Blue', label="Model")

#plt.savefig('Linear_Reg_Poly.png')

CPU times: user 13.3 s, sys: 15.3 s, total: 28.6 s
Wall time: 34 s


Pipeline(steps=[('polynomial_features', PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)), ('linear_regression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [18]:
pipeline.score(X,y)

0.81978845296301617

In [19]:
# MAE:  Mean Absolute mean 
mae = abs(y-pipeline.predict(X)).mean()
print("Mean Absolute mean ",mae)

Mean Absolute mean  420.137739904


In [20]:
# MAE:  Mean Percentage Absolute Error 

# Mean  Percentage  Absolute Error  of linear:  0.128071279637
mpae = (abs(y-pipeline.predict(X))).sum()/y.sum()
print("Mean  Percentage  Absolute Error ",mpae)

Mean  Percentage  Absolute Error  0.207264784072


# Use LinearRegression to train data set with weather feature 

In [21]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [22]:
# Prepare the training features with rain and target feature data set 
feature_cols = ['Distance','midweek','HourOfDay','rain']
X = df[feature_cols]
y = df['Trip_Time']
X.columns

Index(['Distance', 'midweek', 'HourOfDay', 'rain'], dtype='object')

In [23]:
y.shape

(16852100,)

In [24]:
X.dtypes

Distance       int64
midweek        int64
HourOfDay      int64
rain         float64
dtype: object

In [25]:
# Train the data set with PolynomialFeatures when degree = 2 linear regression 
polynomial_features = PolynomialFeatures(degree=2,include_bias=False)
linear_regression = LinearRegression()
pipeline = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
%time pipeline.fit(X, y)

#df.plot(kind='scatter', x='Distance', y='Trip_Time',label="Samples")
#plt.plot(X['Distance'], pipeline.predict(X), c='Blue', label="Model")

#plt.savefig('Linear_Reg_Poly.png')

CPU times: user 25.4 s, sys: 46.8 s, total: 1min 12s
Wall time: 1min 49s


Pipeline(steps=[('polynomial_features', PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)), ('linear_regression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [26]:
%time pipeline.score(X,y)

CPU times: user 8.11 s, sys: 9.24 s, total: 17.4 s
Wall time: 21.3 s


0.81989995394692983

In [27]:
# MAE:  Mean Absolute mean 
mae = abs(y-pipeline.predict(X)).mean()
print("Mean Absolute mean ",mae)

Mean Absolute mean  420.074240134


In [28]:
# MAE:  Mean Percentage Absolute Error 

# Mean  Percentage  Absolute Error  of linear:  0.128071279637
mpae = (abs(y-pipeline.predict(X))).sum()/y.sum()
print("Mean  Percentage  Absolute Error ",mpae)

Mean  Percentage  Absolute Error  0.207233457998


In [23]:
import pickle
pickle.dump(pipeline, open('linear_model_all', 'wb'))

# Train  Random Forest model with train data from all data set 

In [30]:
#from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.cross_validation import train_test_split
#from sklearn.preprocessing import Imputer
#from sklearn.multioutput import MultiOutputRegressor

/Users/April/anaconda/envs/comp47350/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [31]:
# http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
#sklearn.ensemble.RandomForestRegressor
regr_rf = RandomForestRegressor()

In [32]:
# Prepare the training features and target feature data set 
feature_cols = ['Distance','midweek','HourOfDay','rain']
X = df[feature_cols]
y = df['Trip_Time']
X.columns

Index(['Distance', 'midweek', 'HourOfDay', 'rain'], dtype='object')

In [33]:
# Split the data set to 
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.5)

In [34]:
# Check the training data set size 
x_train.shape

(5055630, 4)

In [35]:
#Train the model with train data set 
%time regr_rf.fit(x_train,y_train)

CPU times: user 3min 20s, sys: 3.31 s, total: 3min 23s
Wall time: 3min 26s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [36]:
# Chcek the score of the model with training data set 
%time regr_rf.score(x_train,y_train)

CPU times: user 54.5 s, sys: 5.74 s, total: 1min
Wall time: 1min 3s


0.90439682386864706

In [37]:
# Chcek the score of the model with valation data set 
%time regr_rf.score(x_val,y_val)

CPU times: user 55.4 s, sys: 5.68 s, total: 1min 1s
Wall time: 1min 3s


0.79933668804876612

In [38]:
# MAE:  Mean Absolute Error of testing data set 

%time mae = abs(y_train-regr_rf.predict(x_train)).mean()
print("Mean Absolute Error of train data  ",mae)

CPU times: user 49.8 s, sys: 4.97 s, total: 54.8 s
Wall time: 56.1 s
Mean Absolute Error of train data   287.32434946


In [39]:
# MAE:  Mean Absolute Error of testing data set 

%time mae = abs(y_val-regr_rf.predict(x_val)).mean()
print("Mean Absolute Error of test data  ",mae)

CPU times: user 50.8 s, sys: 4.79 s, total: 55.6 s
Wall time: 56.9 s
Mean Absolute Error of test data   433.05883205


In [41]:
# MAE:  Mean Percentage Absolute Error 

# Mean  Percentage  Absolute Error  of linear:  0.128071279637
mpae = (abs(y_test-regr_rf.predict(x_test))).sum()/y.sum()
print("Mean  Percentage  Absolute Error ",mpae)

Mean  Percentage  Absolute Error  0.0854988376894


In [42]:
import pickle
%time pickle.dump(regr_rf, open('rfmodle.sav', 'wb'))

CPU times: user 2.59 s, sys: 19.5 s, total: 22.1 s
Wall time: 1min 3s


# Split our data to train and test data set and check our model result 


# Linear Regression 

In [43]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

In [44]:
# Prepare the training features and target feature data set 
feature_cols = ['Distance','midweek','HourOfDay','rain']
X = df[feature_cols]
y = df['Trip_Time']
X.columns

Index(['Distance', 'midweek', 'HourOfDay', 'rain'], dtype='object')

In [45]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [46]:
# Train the data set with PolynomialFeatures when degree = 2 linear regression 
polynomial_features = PolynomialFeatures(degree=2,include_bias=False)
linear_regression = LinearRegression()
pipeline_train = Pipeline([("polynomial_features", polynomial_features),
                         ("linear_regression", linear_regression)])
%time pipeline_train.fit(X_train, y_train)

CPU times: user 16.9 s, sys: 27.8 s, total: 44.6 s
Wall time: 1min 9s


Pipeline(steps=[('polynomial_features', PolynomialFeatures(degree=2, include_bias=False, interaction_only=False)), ('linear_regression', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

In [47]:
# Check the socre of the modeli with the training data set 
pipeline_train.score(X_train,y_train)

0.81988157353431679

In [48]:
# Check the socre of the modeli with the testing data set 
pipeline_train.score(X_test,y_test)

0.81994266662008863

In [49]:
# MAE:  Mean Absolute Error  of Training Data 

mae = abs(y_train-pipeline_train.predict(X_train)).mean()
print("Mean Absolute Error of train data set  ",mae)

Mean Absolute Error of train data set   420.101920021


In [50]:
# MAE:  Mean Absolute Error of Testing Data 
 
mae = abs(y_test-pipeline_train.predict(X_test)).mean()
print("Mean Absolute Error of testing data set  ",mae)

Mean Absolute Error of testing data set   419.975907999


## Result of modeling 

Linear regression has the same mean absolute error with the training and testing data set 

Random Forest regressor couldn't handle such big data set, so we split it to small one to train the model. While the random forest's model size is too big, the best way is still to train a model for every Journey_Pattern_ID. 

## Read models with pickle and check the prediction 

In [51]:
# Use pickle the read the model 
import pickle 
filename_all='linear_model_all'
filename='linear_model'
loaded_model_all = pickle.load(open(filename_all, 'rb'))
loaded_model = pickle.load(open(filename, 'rb'))

In [52]:
# Predict the result 
loaded_model.predict([100,1,12])

/Users/April/anaconda/envs/comp47350/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 196.47617497])

In [55]:
# Predict the result 
loaded_model.predict([1000,1,12])

/Users/April/anaconda/envs/comp47350/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 363.53655189])

In [58]:
# Predict the result 
loaded_model_all.predict([1000,1,12,0.2])

/Users/April/anaconda/envs/comp47350/lib/python3.6/site-packages/sklearn/utils/validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)


array([ 365.59216842])